#Prediction Tendency Task

I based my code on this preprint by Juliane Schubert et al. (2022):
https://www.biorxiv.org/content/10.1101/2022.04.22.489224v1.full.pdf

Additionally, Juliane was so kind as to give me access to a repo containing a lot scripts for the implementation of the paradigm in MATLAB.

In [ ]:
""" SETTINGS """

# import modules
import numpy as np
import pandas as pd
import random
import time
import matplotlib.pyplot as plt # for testing

# for hidden markov models:
!pip install hmmlearn
from hmmlearn import hmm

# ------------------------------------

# Settings for the prediction tendency task:

# for the sounds:
tones = [440, 587, 782, 1043]  # Pure tone frequencies in Hz
#tone_duration = 0.1  # Duration of each pure tone in seconds (each lasted 100 ms)
#tone_rate = 3  # Rate of pure tone presentation in Hz
#tone_volume = 1 # use full volume, but you can adjust this later if you determined a hearing threshold
#audio_sample_freq = 44100 # 44100 Hz --> audio sampling rate at the lab (according to Frauke)
#tones_iti = 1/3
#tone_fade = 5e-3

# for the paradigm:
block_trials = 1505  # Number of trials per entropy condition
#trigger_ordered = 1
#trigger_random = 2

In [69]:
# Function for testing if the generation functions actually work as
# intended and produce the correct matrices:

''' Function for calculating transition probabilities of 1 sequence'''
def calculate_transition_probabilities(sequence):
    transition_counts = np.zeros((len(tones), len(tones)))

    for i in range(1, len(sequence)):
        prev_tone = sequence[i-1]
        current_tone = sequence[i]
        prev_tone_index = tones.index(prev_tone)
        current_tone_index = tones.index(current_tone)

        transition_counts[prev_tone_index][current_tone_index] += 1

    # Replace zero counts with a small value
    transition_counts[transition_counts == 0] = 1e-6

    transition_probs = transition_counts / np.sum(transition_counts, axis=1, keepdims=True)
    #print(transition_counts)
    return transition_probs

In [118]:
''' Generate Tone Sequence for the Ordered Condition '''

def generate_ordered_seq(tones, len_sequence):

  # collect tones here:
  final_sequence = []

  # check how many transitions we need for each cell:
  # careful, not tones, but transitions, so subtract 1 from sequence length
  n_transitions_25 = int(( (len_sequence - 1) / 4) / 100 * 25)
  n_transitions_75 = int(( (len_sequence - 1) / 4) / 100 * 75)

  print("25%:", n_transitions_25, "75%:", n_transitions_75)

  # Now generate small transition snippets: We can either go up (e.g. 1 - 2 or 0 - 1)
  # or we can repeat the same tone again (e.g. 3 - 3).
  # For each tone, we need 75% of "up" transitions and 25% of "rep" transitions:
  up_0 = [[0,1]] * n_transitions_75
  rep_0 = [[0,0]] * n_transitions_25
  # combine & shuffle them so we have all transitions that start with 0 in one list:
  all_0 = up_0 + rep_0
  random.shuffle(all_0)
  #print(all_0)

  # Do the same for the other tones' transition lists:
  up_1 = [[1,2]] * n_transitions_75
  rep_1 = [[1,1]] * n_transitions_25
  all_1 = up_1 + rep_1
  random.shuffle(all_1)

  up_2 = [[2,3]] * n_transitions_75
  rep_2 = [[2,2]] * n_transitions_25
  all_2 = up_2 + rep_2
  random.shuffle(all_2)

  up_3 = [[3,0]] * n_transitions_75
  rep_3 = [[3,3]] * n_transitions_25
  all_3 = up_3 + rep_3
  random.shuffle(all_3)

  # put them into a dictionary for easier handling:
  transition_lists = {0: all_0,
                      1: all_1,
                      2: all_2,
                      3: all_3}

  # Idea:
  # I will start with a random tone and pick one transition from the corresponding list (e.g. if I pick 1 I need to take a small transition from all_1)
  # That's our first small transition that we'll put into the final sequence.
  # Then I will remove transition I picked from the list (e.g. remove one [1,2] list from all_1).

  # Next I will take the last element from the transition I picked and randomly draw a transition from the corresponding list.
  # E.g. if my last transition was [1,2], I need to pick a transition from all_2
  curr_idx = random.choice(range(len(tones)))
  print("starting with tone at idx", curr_idx)

  # Now create a loop that only ends if we reach a dead end (e.g. if remaining transitions
  # don't match curr_idx anymore, I think such a scenario might be possible)
  # or if all our lists are empty.

  while True:

    # if there are still transitions left for us to pick:
    if len(transition_lists[curr_idx]) > 0:

      # choose a transition for the current tone:
      curr_trans = random.choice(transition_lists[curr_idx])

      # put the transition into the final sequence:
      final_sequence.extend([curr_trans])

      # remove 1 matching transition from the list we got curr_trans from:
      transition_lists[curr_idx].remove(curr_trans)

      # get last element from the transition so we can pick the next transition:
      curr_idx = curr_trans[1]

    else:
      # If we reached a dead end and there are still transitions to pick:
      if any(len(transition_list) > 0 for transition_list in transition_lists.values()):

        print("Reached a dead end! Using backtracking now!")
        # Remove the last 20 transitions from the final sequence
        for _ in range(200):

          # get last transition list from final_sequence
          last_trans = final_sequence.pop(len(final_sequence)-1)
          prev_idx = last_trans[0]

          # Add the removed transition back to the list so it can be picked again
          transition_lists[prev_idx].append(last_trans)
          #print("removed transition")

        # Start over with new transition (depending on last transition in final_sequence)
        curr_idx = final_sequence[-1][1]

      else:
        # All transitions have been used, exit the loop
        break

  #print(len(final_sequence))
  #print(final_sequence)

  # Now as you can see, we need to shorten the sequence a bit because we still only have the transitions, not single tones.
  # So we want to go from this: [1,1], [1,2], [2,3], [3,0] to this: [1, 1, 2, 3, 0]
  # As you can see we need to keep the values from the first transition and only the second value from all others.
  sequence = final_sequence[0]
  # remove first transition from final_sequence:
  final_sequence.pop(0)
  # loop the rest of the transitions and always append the second value from each transition to the sequence:
  for transition in final_sequence:
    sequence.append(transition[1])

  # Now turn 0, 1, 2 and 3 into corresponding frequencies in list tones:
  sequence = [tones[idx] for idx in sequence]

  return(sequence)


# Test case:

tones = [440, 587, 782, 1043]  # Pure tone frequencies in Hz
len_sequence = 1505 # careful, I need a non-float number of transitions for the 75% and 25% cells, so I need 1505 values

sequence = generate_ordered_seq(tones, len_sequence)

print("------------------")

# Check our transition probability matrix
print(calculate_transition_probabilities(sequence))

print("------------------")

# Check how many occurances we have for each tone:
print("count of freq. 440 Hz in sequence:", sequence.count(440))
print("count of freq. 587 Hz in sequence:", sequence.count(587))
print("count of freq. 782 Hz in sequence:", sequence.count(782))
print("count of freq. 1043 Hz in sequence:", sequence.count(1043))

print("------------------")

# Check length of sequence:
print("length of generated sequence:", len(sequence))
# Okay so we have 1 occurance too much for one tone, but that's okay I guess.
# We'll probably have to exclude some trials in the analysis anyway.

print("sequence:", sequence)

25%: 94 75%: 282
starting with tone at idx 2
Reached a dead end! Using backtracking now!
------------------
[[0.25 0.75 0.   0.  ]
 [0.   0.25 0.75 0.  ]
 [0.   0.   0.25 0.75]
 [0.75 0.   0.   0.25]]
------------------
count of freq. 440 Hz in sequence: 376
count of freq. 587 Hz in sequence: 376
count of freq. 782 Hz in sequence: 377
count of freq. 1043 Hz in sequence: 376
------------------
length of generated sequence: 1505
sequence: [782, 1043, 440, 587, 782, 1043, 440, 587, 782, 782, 1043, 440, 587, 782, 782, 782, 1043, 440, 587, 782, 1043, 440, 587, 587, 587, 782, 782, 782, 1043, 440, 587, 587, 782, 1043, 440, 587, 782, 1043, 440, 440, 587, 587, 782, 1043, 440, 440, 587, 782, 782, 1043, 440, 440, 587, 587, 587, 782, 1043, 440, 440, 587, 782, 1043, 440, 440, 440, 587, 782, 782, 782, 1043, 440, 587, 782, 782, 782, 1043, 440, 587, 782, 1043, 440, 587, 782, 1043, 1043, 440, 440, 587, 587, 782, 1043, 440, 440, 587, 782, 1043, 440, 440, 587, 782, 1043, 440, 587, 782, 1043, 440, 440, 44

In [ ]:
''' Generate Tone Sequence for the Ordered Condition '''

def generate_random_seq(tones, len_sequence):

  # collect tones here:
  final_sequence = []

  # check how many transitions we need for each cell:
  # careful, not tones, but transitions, so subtract 1 from sequence length
  n_transitions_25 = int(( (len_sequence - 1) / 4) / 100 * 25)

  print("25%:", n_transitions_25)

  # Now generate small transition snippets: We can either go up (e.g. 1 - 2 or 0 - 1)
  # or we can repeat the same tone again (e.g. 3 - 3).
  # For each tone, we need 75% of "up" transitions and 25% of "rep" transitions:
  trans0_0 = [[0,0]] * n_transitions_25
  trans1_0 = [[0,1]] * n_transitions_25
  trans2_0 = [[0,2]] * n_transitions_25
  trans3_0 = [[0,3]] * n_transitions_25
  # combine & shuffle them so we have all transitions that start with 0 in one list:
  all_0 = trans0_0 + trans1_0 + trans2_0 + trans3_0
  random.shuffle(all_0)
  #print(all_0)

  # Do the same for the other tones' transition lists:
  trans0_1 = [[1,0]] * n_transitions_25
  trans1_1 = [[1,1]] * n_transitions_25
  trans2_1 = [[1,2]] * n_transitions_25
  trans3_1 = [[1,3]] * n_transitions_25
  all_1 = trans0_1 + trans1_1 + trans2_1 + trans3_1
  random.shuffle(all_1)

  trans0_2 = [[2,0]] * n_transitions_25
  trans1_2 = [[2,1]] * n_transitions_25
  trans2_2 = [[2,2]] * n_transitions_25
  trans3_2 = [[2,3]] * n_transitions_25
  all_2 = trans0_2 + trans1_2 + trans2_2 + trans3_2
  random.shuffle(all_2)

  trans0_3 = [[3,0]] * n_transitions_25
  trans1_3 = [[3,1]] * n_transitions_25
  trans2_3 = [[3,2]] * n_transitions_25
  trans3_3 = [[3,3]] * n_transitions_25
  all_3 = trans0_3 + trans1_3 + trans2_3 + trans3_3
  random.shuffle(all_3)

  # put them into a dictionary for easier handling:
  transition_lists = {0: all_0,
                      1: all_1,
                      2: all_2,
                      3: all_3}

  # Idea:
  # I will start with a random tone and pick one transition from the corresponding list (e.g. if I pick 1 I need to take a small transition from all_1)
  # That's our first small transition that we'll put into the final sequence.
  # Then I will remove transition I picked from the list (e.g. remove one [1,2] list from all_1).

  # Next I will take the last element from the transition I picked and randomly draw a transition from the corresponding list.
  # E.g. if my last transition was [1,2], I need to pick a transition from all_2
  curr_idx = random.choice(range(len(tones)))
  print("starting with tone at idx", curr_idx)

  # Now create a loop that only ends if we reach a dead end (e.g. if remaining transitions
  # don't match curr_idx anymore, I think such a scenario might be possible)
  # or if all our lists are empty.

  while True:

    # if there are still transitions left for us to pick:
    if len(transition_lists[curr_idx]) > 0:

      # choose a transition for the current tone:
      curr_trans = random.choice(transition_lists[curr_idx])

      # put the transition into the final sequence:
      final_sequence.extend([curr_trans])

      # remove 1 matching transition from the list we got curr_trans from:
      transition_lists[curr_idx].remove(curr_trans)

      # get last element from the transition so we can pick the next transition:
      curr_idx = curr_trans[1]

    else:
      # If we reached a dead end and there are still transitions to pick:
      if any(len(transition_list) > 0 for transition_list in transition_lists.values()):

        print("Reached a dead end! Using backtracking now!")
        # Remove the last 20 transitions from the final sequence
        for _ in range(200):

          # get last transition list from final_sequence
          last_trans = final_sequence.pop(len(final_sequence)-1)
          prev_idx = last_trans[0]

          # Add the removed transition back to the list so it can be picked again
          transition_lists[prev_idx].append(last_trans)
          #print("removed transition")

        # Start over with new transition (depending on last transition in final_sequence)
        curr_idx = final_sequence[-1][1]

      else:
        # All transitions have been used, exit the loop
        break

  #print(len(final_sequence))
  #print(final_sequence)

  # Now as you can see, we need to shorten the sequence a bit because we still only have the transitions, not single tones.
  # So we want to go from this: [1,1], [1,2], [2,3], [3,0] to this: [1, 1, 2, 3, 0]
  # As you can see we need to keep the values from the first transition and only the second value from all others.
  sequence = final_sequence[0]
  # remove first transition from final_sequence:
  final_sequence.pop(0)
  # loop the rest of the transitions and always append the second value from each transition to the sequence:
  for transition in final_sequence:
    sequence.append(transition[1])

  # Now turn 0, 1, 2 and 3 into corresponding frequencies in list tones:
  sequence = [tones[idx] for idx in sequence]

  return(sequence)


# Test case:

tones = [440, 587, 782, 1043]  # Pure tone frequencies in Hz
len_sequence = 1505 # careful, I need a non-float number of transitions for the 75% and 25% cells, so I need 1505 values

sequence = generate_random_seq(tones, len_sequence)

print("------------------")

# Check our transition probability matrix
print(calculate_transition_probabilities(sequence))

print("------------------")

# Check how many occurances we have for each tone:
print("count of freq. 440 Hz in sequence:", sequence.count(440))
print("count of freq. 587 Hz in sequence:", sequence.count(587))
print("count of freq. 782 Hz in sequence:", sequence.count(782))
print("count of freq. 1043 Hz in sequence:", sequence.count(1043))

print("------------------")

# Check length of sequence:
print("length of generated sequence:", len(sequence))
# Okay so we have 1 occurance too much for one tone, but that's okay I guess.
# We'll probably have to exclude some trials in the analysis anyway.

print("sequence:", sequence)

In [122]:
# prepare 1000 sound sequences for both conditions
 # (so that basically I could run this study until I'm 80
 # and won't run out of sequences)
import pandas as pd


tones = [440, 587, 782, 1043]  # Pure tone frequencies in Hz
len_sequence = 1505 # careful, I need a non-float number of transitions for the 75% and 25% cells, so I need 1505 values

# Create empty data frames for both conditions:
df_ordered_seqs = pd.DataFrame()
df_random_seqs = pd.DataFrame()

# do this 1000x
for i in range(1000):
  print(i)
  # Generate the trial sequences for both entropy conditions
  ordered_sequence = generate_ordered_seq(tones, len_sequence)
  random_sequence = generate_random_seq(tones, len_sequence)

  # append current arrays as new rows to the 2 data frames
  df_ordered_seqs = df_ordered_seqs.append(pd.Series(ordered_sequence), ignore_index = True)
  df_random_seqs = df_random_seqs.append(pd.Series(random_sequence), ignore_index = True)



0
25%: 94 75%: 282
starting with tone at idx 3
25%: 94
starting with tone at idx 1
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
1
25%: 94 75%: 282
starting with tone at idx 1
25%: 94
starting with tone at idx 1
2
25%: 94 75%: 282
starting with tone at idx 2
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
25%: 94
starting with tone at idx 1
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
3
2

<ipython-input-122-f653ad1abf66>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ordered_seqs = df_ordered_seqs.append(pd.Series(ordered_sequence), ignore_index = True)
<ipython-input-122-f653ad1abf66>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_random_seqs = df_random_seqs.append(pd.Series(random_sequence), ignore_index = True)
<ipython-input-122-f653ad1abf66>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ordered_seqs = df_ordered_seqs.append(pd.Series(ordered_sequence), ignore_index = True)
<ipython-input-122-f653ad1abf66>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_random_seqs = df_random_seqs.append(pd.

13
25%: 94 75%: 282
starting with tone at idx 0
Reached a dead end! Using backtracking now!
25%: 94
starting with tone at idx 3
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
14
25%: 94 75%: 282
starting with tone at idx 2
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
25%: 94
starting with tone at idx 1
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
15
25%: 94 75%: 282
starting with tone a

<ipython-input-122-f653ad1abf66>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ordered_seqs = df_ordered_seqs.append(pd.Series(ordered_sequence), ignore_index = True)
<ipython-input-122-f653ad1abf66>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_random_seqs = df_random_seqs.append(pd.Series(random_sequence), ignore_index = True)
<ipython-input-122-f653ad1abf66>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ordered_seqs = df_ordered_seqs.append(pd.Series(ordered_sequence), ignore_index = True)
<ipython-input-122-f653ad1abf66>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_random_seqs = df_random_seqs.append(pd.

25
25%: 94 75%: 282
starting with tone at idx 0
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
25%: 94
starting with tone at idx 0
26
25%: 94 75%: 282
starting with tone at idx 1
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
25%: 94
starting with tone at idx 2
Reached a dead end! Using backtracking now!
27
25%: 94 75%: 282
starting with tone at idx 1
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
25%: 94
starting with tone at idx 1
28
25%: 94 75%: 282
starting with tone at idx 3
Reached a dead end! Using backtracking now!
25%: 94
starting with tone at idx 2
29
25%: 94 75%: 282
starting with tone at idx 1
25%: 94
starting with tone at idx 2
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached 

...

<ipython-input-122-f653ad1abf66>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_random_seqs = df_random_seqs.append(pd.Series(random_sequence), ignore_index = True)
<ipython-input-122-f653ad1abf66>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ordered_seqs = df_ordered_seqs.append(pd.Series(ordered_sequence), ignore_index = True)
<ipython-input-122-f653ad1abf66>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_random_seqs = df_random_seqs.append(pd.Series(random_sequence), ignore_index = True)
<ipython-input-122-f653ad1abf66>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ordered_seqs = df_ordered_seqs.append(pd.S

 94 75%: 282
starting with tone at idx 1
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
25%: 94
starting with tone at idx 3
Reached a dead end! Using backtracking now!
989
25%: 94 75%: 282
starting with tone at idx 1
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
25%: 94
starting with tone at idx 3
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
990
25%: 94 75%: 282
starting with tone at idx 2
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
25%: 94
starting with tone at idx 3
Reached a dead end! Using backtracking now!
991
25%: 94 75%: 282
starting with tone at idx 0
25%: 94
starting with tone at idx 0
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached 

<ipython-input-122-f653ad1abf66>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_random_seqs = df_random_seqs.append(pd.Series(random_sequence), ignore_index = True)
<ipython-input-122-f653ad1abf66>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ordered_seqs = df_ordered_seqs.append(pd.Series(ordered_sequence), ignore_index = True)
<ipython-input-122-f653ad1abf66>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_random_seqs = df_random_seqs.append(pd.Series(random_sequence), ignore_index = True)
<ipython-input-122-f653ad1abf66>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ordered_seqs = df_ordered_seqs.append(pd.S

995
25%: 94 75%: 282
starting with tone at idx 0
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
25%: 94
starting with tone at idx 2
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
Reached a dead end! Using backtracking now!
996
25%: 94 75%: 282
starting with tone at idx 1
Reached a dead end! Using backtracking now!
25%: 94
starting with tone at idx 3
997
25%: 94 75%: 282
starting with tone at idx 0
Reached a dead end! Using backtracking now!
25%: 94
starting with tone at idx 2
Reached a dead end! Using backtracking now!
998
25%: 94 75%: 282
starting with tone at idx 0
Reached a dead end! Using backtracking now!
25%: 94
starting with tone at idx 2
999
25%: 94 75%: 282
starting with tone at i

<ipython-input-122-f653ad1abf66>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_random_seqs = df_random_seqs.append(pd.Series(random_sequence), ignore_index = True)
<ipython-input-122-f653ad1abf66>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ordered_seqs = df_ordered_seqs.append(pd.Series(ordered_sequence), ignore_index = True)
<ipython-input-122-f653ad1abf66>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_random_seqs = df_random_seqs.append(pd.Series(random_sequence), ignore_index = True)
<ipython-input-122-f653ad1abf66>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ordered_seqs = df_ordered_seqs.append(pd.S

In [123]:
# download dataframes
from google.colab import files

df_ordered_seqs.to_csv('df_ordered_seqs.csv', encoding = "utf-8-sig")
df_random_seqs.to_csv('df_random_seqs.csv', encoding = "utf-8-sig")
files.download('df_ordered_seqs.csv')
files.download('df_random_seqs.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

------------------------------------------------

OLD SNIPPETS:


In [105]:
# Works but works with probabilities (so it's not exact) and not with fixed frequencies

def generate_trial_sequence(entropy_level):

    # check how many tones we have - this determines
    # how big our matrices with the transition probabilities
    # need to be:
    n_features = len(tones)

    # create emission matrix: We want to emit
    # the given state with 100% probability
    # (feels a bit odd to explicitly specify that,
    # but we need it for the HMM function later)
    emission_matrix = np.eye(n_features)
    #print(emission_matrix)


    if entropy_level == "ordered":
        # create a matrix with transition probabilities
        # for the ordered condition:
        transition_matrix = [[0.25, 0.75, 0., 0.],
                             [0., 0.25, 0.75, 0.],
                             [0., 0., 0.25, 0.75],
                             [0.75, 0., 0., 0.25]]
        #print("ordered transition probabilities matrix:", transition_matrix)

    elif entropy_level == "random":
        # create a matrix with transition probabilities
        # for the random condition:
        transition_matrix = [[0.25, 0.25, 0.25, 0.25],
                             [0.25, 0.25, 0.25, 0.25],
                             [0.25, 0.25, 0.25, 0.25],
                             [0.25, 0.25, 0.25, 0.25]]
        #print("random transition probabilities matrix:", transition_matrix)




    # Create an HMM model
    model = hmm.MultinomialHMM(n_components = n_features,
                               n_trials = 1,
                               verbose = False)

    model.startprob_ = np.ones(n_features) / n_features  # Set equal initial probabilities
    model.transmat_ = transition_matrix
    model.emissionprob_ = emission_matrix

    # Generate a sequence of pure tones
    sequence, _ = model.sample(n_samples = 1500, random_state = None)

    pure_tones_sequence = []

    for sublist in sequence:
      #print(sublist)
      index = np.where(sublist != 0)[0][0]  # Get the index of the non-zero element
      #print(index)
      pure_tone = tones[index]  # Retrieve the corresponding pure tone
      pure_tones_sequence.append(pure_tone)

    # TO DO:
    # The transition probabilities don't match exactly the probabilities I specified in my transition matrix.
    # Idea: Polish it a bit manually, so I get exactly 0%, 25% and 75% transition probabilities in the
    # output? Or is this not a problem?

    return(pure_tones_sequence)

# Generate the trial sequences for both entropy conditions
ordered_sequence = generate_trial_sequence("ordered")
random_sequence = generate_trial_sequence("random")
print(ordered_sequence)

calculate_transition_probabilities(ordered_sequence)

In [ ]:
# also works, but this is for the snippet above:

'''
Generate 10,000 sequences for each condition and
test their transition probabilities
'''
# Create empty data frames for both conditions:
df_ordered_test = pd.DataFrame()
df_random_test = pd.DataFrame()

# do this 10,000x
for i in range(10000):
  print(i)
  # Generate the trial sequences for both entropy conditions
  ordered_sequence = generate_trial_sequence("ordered")
  random_sequence = generate_trial_sequence("random")

  # append current arrays as new rows to the 2 data frames
  df_ordered_test = df_ordered_test.append(pd.Series(ordered_sequence), ignore_index = True)
  df_random_test = df_random_test.append(pd.Series(random_sequence), ignore_index = True)


# tones was defined above like this:
#tones = [440, 587, 782, 1043]

# create 2 empty average transition probability
# matrices for our 2 conditions:
avg_transition_probs_ordered = np.zeros((len(tones), len(tones)))
avg_transition_probs_random = np.zeros((len(tones), len(tones)))

# loop rows in df_ordered_test:
for _, row in df_ordered_test.iterrows():
    # get current row
    sequence = row.values
    # calculate transition probabilities
    transition_probs = calculate_transition_probabilities(sequence)
    # add matrix values to our matrix with the average values
    avg_transition_probs_ordered += transition_probs

# do the same for the dataframe df_random_test:
for _, row in df_random_test.iterrows():
    sequence = row.values
    transition_probs = calculate_transition_probabilities(sequence)
    avg_transition_probs_random += transition_probs

# Divide values in the matrices by the total number of rows in
# each dataframe to get average transition probabilities
avg_transition_probs_ordered /= len(df_ordered_test)
avg_transition_probs_random /= len(df_random_test)



# Set numpy print options
np.set_printoptions(precision = 3, suppress = True)

# Display the transition probability matrices
print("Average Ordered Transition Probabilities Matrix:")
print(avg_transition_probs_ordered)
print()

# Display the transition probability matrices
print("Average Ordered Transition Probabilities Matrix:")
print(avg_transition_probs_random)
print()

In [106]:
#This gets stuck and doesn't produce the correct matrices:


import random
import numpy as np
# turn off scientific notation:
np.set_printoptions(precision = 6, suppress = True)


# Okay so the logic behind this is that I want a sequence with 4 tones (all occuring equally often) and
# certain transition probabilities of 0, 25 or 75%. So I use 2 kinds of counters, one for the
# number of occurances of each tone and one kind for the transitions.
# I'm basically creating a more controlled HMM sequence with fixed numbers of tones & transitions.


def generate_sequence_from_matrix(condition, stimuli, len_sequence):

    # how many stimuli do we have?
    num_stimuli = len(stimuli)

    # specify transition probability matrices depending on condition:
    if condition == "ordered":

        # start with empty sequence:
        sequence = []

        # check how many transitions we can use for the 25% and 75% cells of the matrices.
        # This basically tells us how often we can choose A or B for example if the current value is A.
        count25_0 = (len_sequence / 4) / 100 * 25
        count75_0 = (len_sequence / 4) / 100 * 75
        count25_1 = (len_sequence / 4) / 100 * 25
        count75_1 = (len_sequence / 4) / 100 * 75
        count25_2 = (len_sequence / 4) / 100 * 25
        count75_2 = (len_sequence / 4) / 100 * 75
        count25_3 = (len_sequence / 4) / 100 * 25
        count75_3 = (len_sequence / 4) / 100 * 75

        # count instances of each stimulus so we make sure there
        # are exactly equal amounts of them in the sequence:
        count_0 = 0
        count_1 = 0
        count_2 = 0
        count_3 = 0

        # as a starting point, choose random stimulus from list of tones:
        current_stimulus = random.choice(stimuli)
        # append current stimulus to the sequence
        sequence.append(current_stimulus)

        # now loop length of sequence you want to generate
        for x in range(len_sequence-1):

            # get idx of current stimulus in the stimulus list
            current_stimulus_index = stimuli.index(current_stimulus)
            #print(current_stimulus_index)

            # STIMULUS 0:
            # If the current stimulus is 0 and we still have enough stimuli left of
            # stimuli 0 and 1, you can choose. If not, take what's left.
            if current_stimulus_index == 0:
              if count_0 < (len_sequence / num_stimuli) and count_1 < (len_sequence / num_stimuli) and count75_0 > 0 and count25_0 > 0:
                # randomly choose next stimulus based on our pre-defined probabilities:
                next_stimulus_index = random.choices([0,1], [.25, .75])[0]
                if next_stimulus_index == 1:
                  count75_0 = count75_0 - 1 # one less transition from 0 to 1 left - update counter!
                else:
                  count25_0 = count25_0 - 1 # one less transition from 0 to 0 left - update counter!
              else:
                # if we can't use stimulus 0 anymore and the transition to 1 is still allowed:
                if count_0 >= (len_sequence / num_stimuli) and count25_0 > 0:
                  #print("counter 0 reached the limit!")
                  next_stimulus_index = 1
                  count75_0 = count75_0 - 1
                # if we can't use stimulus 1 anymore and the transition to 0 is still allowed:
                elif count_1 >= (len_sequence / num_stimuli) and count75_0 > 0:
                  #print("counter 1 reached the limit!")
                  next_stimulus_index = 0
                  count25_0 = count25_0 - 1
                else:
                  print("0: something odd is happening here:", "count 1: ", count_1, "count 0: ", count_0, count75_0, count25_0)
                  # couldn't find solution - recursion!
                  generate_sequence_from_matrix(condition = condition, stimuli = stimuli, len_sequence = len_sequence)
            # STIMULUS 1:
            # If the current stimulus is 1 and we still have enough stimuli left of
            # stimuli 1 and 2, you can choose. If not, take what's left.
            if current_stimulus_index == 1:
              if count_1 < (len_sequence / num_stimuli) and count_2 < (len_sequence / num_stimuli) and count75_1 > 0 and count25_1 > 0:
                # randomly choose next stimulus based on our pre-defined probabilities:
                next_stimulus_index = random.choices([1,2], [.25, .75])[0]
                if next_stimulus_index == 2:
                  count75_1 = count75_1 - 1 # one less transition from 1 to 2 left - update counter!
                else:
                  count25_1 = count25_1 - 1 # one less transition from 1 to 1 left - update counter!
              else:
                # if we can't use stimulus 1 anymore and the transition to 2 is still allowed:
                if count_1 >= (len_sequence / num_stimuli) and count25_1 > 0:
                  next_stimulus_index = 2
                  count75_1 = count75_1 - 1
                  #print("counter 1 reached the limit!")
                # if we can't use stimulus 2 anymore and the transition to 1 is still allowed:
                elif count_2 >= (len_sequence / num_stimuli) and count75_1 > 0:
                  #print("counter 2 reached the limit!")
                  count25_1 = count25_1 - 1
                  next_stimulus_index = 1
                else:
                  print("1: something odd is happening here:", "count 2: ", count_2, "count 1: ", count_1, count75_1, count25_1)
                  # couldn't find solution - recursion!
                  generate_sequence_from_matrix(condition = condition, stimuli = stimuli, len_sequence = len_sequence)
            # STIMULUS 2:
            # If the current stimulus is 2 and we still have enough stimuli left of
            # stimuli 2 and 3, you can choose. If not, take what's left.
            if current_stimulus_index == 2:
              if count_2 < (len_sequence / num_stimuli) and count_3 < (len_sequence / num_stimuli) and count75_2 > 0 and count25_2 > 0 :
                # randomly choose next stimulus based on our pre-defined probabilities:
                next_stimulus_index = random.choices([2,3], [.25, .75])[0]
                if next_stimulus_index == 3:
                  count75_2 = count75_2 - 1 # one less transition from 2 to 3 left - update counter!
                else:
                  count25_2 = count25_2 - 1 # one less transition from 2 to 2 left - update counter!
              # if at least one of the options can't be used anymore:
              else:
                # if we can't use stimulus 2 anymore and the transition to 3 is still allowed:
                if count_2 >= (len_sequence / num_stimuli) and count75_2 > 0:
                  #print("counter 2 reached the limit!")
                  count75_2 = count75_2 - 1  # one less transition from 2 to 3 left - update counter!
                  next_stimulus_index = 3
                # if we can't use stimulus 3 anymore and the transition to 2 is still allowed:
                elif count_3 >= (len_sequence / num_stimuli) and count25_2 > 0:
                  #print("counter 3 reached the limit!")
                  count25_2 = count25_2 - 1  # one less transition from 2 to 2 left - update counter!
                  next_stimulus_index = 2
                else:
                  print("2: something odd is happening here:", "count 3: ", count_3, "count 2: ", count_2, count75_2, count25_2)
                  # couldn't find solution - recursion!
                  generate_sequence_from_matrix(condition = condition, stimuli = stimuli, len_sequence = len_sequence)
            # STIMULUS 3:
            # If the current stimulus is 3 and we still have enough stimuli left of
            # stimuli 3 and 0, you can choose. If not, take what's left.
            if current_stimulus_index == 3:
              if count_3 < (len_sequence / num_stimuli) and count_0 < (len_sequence / num_stimuli) and count75_3 > 0 and count25_3 > 0:
                # randomly choose next stimulus based on our pre-defined probabilities:
                next_stimulus_index = random.choices([3,0], [.25, .75])[0]
                if next_stimulus_index == 0:
                  count75_3 = count75_3 - 1 # one less transition from 3 to 0 left - update counter!
                else:
                  count25_3 = count25_3 - 1 # one less transition from 3 to 3 left - update counter!
              else:
                # if we can't use stimulus 3 anymore and the transition to 0 is still allowed:
                if count_3 >= (len_sequence / num_stimuli) and count25_3 > 0:
                  #print("counter 3 reached the limit!")
                  next_stimulus_index = 0
                  count75_3 = count75_3 - 1  # one less transition from 3 to 0 left - update counter!
                # if we can't use stimulus 0 anymore and the transition to 3 is still allowed:
                elif count_0 >= (len_sequence / num_stimuli) and count75_3 > 0:
                  #print("counter 0 reached the limit!")
                  next_stimulus_index = 3
                  count25_3 = count25_3 - 1  # one less transition from 3 to 3 left - update counter!
                else:
                  print("3: something odd is happening here:", "count 3:", count_3, "count 0:", count_0, "75:", count75_3, "25:", count25_3)
                  # couldn't find solution - recursion!
                  generate_sequence_from_matrix(condition = condition, stimuli = stimuli, len_sequence = len_sequence)
            # update counter:
            if next_stimulus_index == 0:
              count_0 = count_0 + 1
            elif next_stimulus_index == 1:
              count_1 = count_1 + 1
            elif next_stimulus_index == 2:
              count_2 = count_2 + 1
            elif next_stimulus_index == 3:
              count_3 = count_3 + 1

            # update current stimulus based on next_stimulus_index
            current_stimulus = stimuli[next_stimulus_index]

            # append current stimulus to the sequence
            sequence.append(current_stimulus)
        print(len_sequence / num_stimuli)
        return sequence

    elif condition == "random":
      pass


tones = [440, 587, 782, 1043]
sequence_length = 1500

result_sequence = generate_sequence_from_matrix(condition = "ordered",stimuli = tones, len_sequence = 1504)
# --> with 1500 stimuli and 4 stimuli, you will never have non-float numbers for the absolute numbers of transitions, so
# by definition, the transition probabilities CAN'T match the 25% and 75% if we don't use
#a little more or a little less values.

#print(calculate_transition_probabilities(result_sequence))

print(result_sequence.count(440))
print(result_sequence.count(587))
print(result_sequence.count(782))
print(result_sequence.count(1043))

print(result_sequence.count(440) + result_sequence.count(587) + result_sequence.count(782) + result_sequence.count(1043))

3: something odd is happening here: count 3: 362 count 0: 346 75: 15.0 25: 0.0
0: something odd is happening here: count 1:  312 count 0:  325 51.0 0.0
0: something odd is happening here: count 1:  304 count 0:  318 59.0 0.0
3: something odd is happening here: count 3: 330 count 0: 325 75: 47.0 25: 0.0
2: something odd is happening here: count 3:  301 count 2:  311 66.0 0.0
0: something odd is happening here: count 1:  332 count 0:  339 38.0 0.0
3: something odd is happening here: count 3: 326 count 0: 285 75: 50.0 25: 0.0
3: something odd is happening here: count 3: 355 count 0: 331 75: 22.0 25: 0.0
0: something odd is happening here: count 1:  306 count 0:  328 48.0 0.0
0: something odd is happening here: count 1:  352 count 0:  357 20.0 0.0
1: something odd is happening here: count 2:  342 count 1:  344 33.0 0.0
0: something odd is happening here: count 1:  364 count 0:  353 0.0 23.0
3: something odd is happening here: count 3: 325 count 0: 309 75: 52.0 25: 0.0
1: something odd is h

KeyboardInterrupt: ignored

In [104]:
# this doesn't work without PsychoPy's sound module

''' PREP SOUNDS FOR RANDOM SEQUENCE: '''

# create an empty dict to store the prepared sounds
tones_random = {}

# loop over list random_sequence with all frequencies
for tone_idx, curr_freq in enumerate(random_sequence):

    print("curent frequency:", curr_freq)

    # build a time array: you need the sound duration and the right sampling frequency for your device
    # 1 divided by the sampling rate = duration of a single sample in sec
    tone_sample_len = 1/audio_sample_freq
    t = np.arange(0, tone_duration, tone_sample_len)

    # generate sine wave:
    sine_wave = np.sin(2*np.pi*curr_freq*t)

    # plot the sine wave
    plt.plot(t, sine_wave)
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.show()

    # Apply cosine ramp to "smoothen" the edges of the sound a bit (I'm not an audio expert as you can tell)
    # We basically gradually turn up the sound, play it for a while,
    # and then decrease the volume again so it doesn't make annoying clicky noises when it's played.

    # apply cosine ramp:
    # check how many samples we have to use for the fade in/out:
    fade_samples = int(tone_fade * audio_sample_freq)

    # if there are enough, but not too many fade samples,
    # apply cosine ramp to signal
    if fade_samples > 0 and fade_samples < len(sine_wave):
      ramp = np.cos(np.linspace(0, np.pi / 2, fade_samples))
      sine_wave[:fade_samples] *= ramp[::-1]
      sine_wave[-fade_samples:] *= ramp

    # plot the modified sine wave again
    plt.plot(t, sine_wave)
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.show()

    print("----------------------")


    # generate sound object for the sound file we built
    sound = Sound(value = sine_wave,
                  secs = tone_duration, # duration of sound in seconds
                  sampleRate = sRate,
                  name = f"tone{tone_idx + 1}", # create a name for the sound for logging
                  hamming = False, # don't apply filter, we did this before
                  volume = tone_volume,
                  loops = 0) # don't repeat sound, play only once

    # add the sound to the dict
    tones_random[f"tone{tone_idx + 1}_random"] = sound





''' PREP SOUNDS FOR ORDERED SEQUENCE: '''

# create an empty dict to store the prepared sounds
tones_ordered = {}

# loop over list ordered_sequence with all frequencies
for tone_idx, curr_freq in enumerate(ordered_sequence):

    print("curent frequency:", curr_freq)

    # build a time array: you need the sound duration and the right sampling frequency for your device
    # 1 divided by the sampling rate = duration of a single sample in sec
    tone_sample_len = 1/audio_sample_freq
    t = np.arange(0, tone_duration, tone_sample_len)

    # generate sine wave:
    sine_wave = np.sin(2*np.pi*curr_freq*t)

    # plot the sine wave
    plt.plot(t, sine_wave)
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.show()

    # Apply cosine ramp to "smoothen" the edges of the sound a bit (I'm not an audio expert as you can tell)
    # We basically gradually turn up the sound, play it for a while,
    # and then decrease the volume again so it doesn't make annoying clicky noises when it's played.

    # apply cosine ramp:
    # check how many samples we have to use for the fade in/out:
    fade_samples = int(tone_fade * audio_sample_freq)

    # if there are enough, but not too many fade samples,
    # apply cosine ramp to signal
    if fade_samples > 0 and fade_samples < len(sine_wave):
      ramp = np.cos(np.linspace(0, np.pi / 2, fade_samples))
      sine_wave[:fade_samples] *= ramp[::-1]
      sine_wave[-fade_samples:] *= ramp

    # plot the modified sine wave again
    plt.plot(t, sine_wave)
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.show()

    print("----------------------")


    # generate sound object for the sound file we built
    sound = Sound(value = sine_wave,
                  secs = tone_duration, # duration of sound in seconds
                  sampleRate = sRate,
                  name = f"tone{tone_idx + 1}", # create a name for the sound for logging
                  hamming = False, # don't apply filter, we did this before
                  volume = tone_volume,
                  loops = 0) # don't repeat sound, play only once

    # add the sound to the dict
    tones_ordered[f"tone{tone_idx + 1}_ordered"] = sound

# now you can access & play each sound by its name, like this:
#now = ptb.GetSecs()
#tones_ordered["tone1_ordered"].play(when = now)  # Play the first sound immediately
#core.wait(1.5) # wait until sound is finished

NameError: ignored